## Demo: SingleRun with HasDelayPlugin
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

This example notebook shows how you can integrate percentual delays in your simulation. We demonstrate how you can use the HasDelayPlugin, and how you can give it a certain delay percentage.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd
import numpy as np

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugins
import opentnsim.core

# TO DO
# x Length at waterline and width at waterline (CALculate hull speed of vessel)->(L_wl = 159)
# x P full and empty correct difference (Next meeting actual T calculation -> Calculations run, but incorrect, why?)(Draft loaded is 13.55, draft empty = 5.90)
# x compare Man Jiang results with mine, see Mark his message (Next week together)
# x Add resistance for seagoing (Use V_2 or V_0?)( Make if statement if shallow water, V_2 or V_0)-> V_2 is applied for Residual and wave resistance

# P moet niet hoger worden dan P installed, dan past de snelhied aan aan P installed.(P is way too high because of R_w, Katsui. Have a look.)
# Compare estimate with VOHOP

# See chat for power to velocity
# insert acceleration with power2v function. Better implementation for calculate power
# One notebook for calculating velocity depending on a set power. From 0 to end. Need to know resistance over time for increasing velocity.
# Depth integration. Varying depth over the graph -> see M v koningsveld. Might connect with opentnsim. move function -> velocity will be connected together.

In [2]:
#pip list
# https://www.dredgepoint.org/dredging-database/equipment/ham-318 (L_wl = 159)

In [3]:
def calculate_power(V_0=7.92, h=70,
                    L_wl=159, B=32, T = 13.55, T_e=1, T_f = 13.55,
                    nu=0.000001, C_b=0.85, rho=1025,
                    c_stern = 1, one_k2=2.5, g=9.81, P_inst = 28636, P_max = 25200, V_max_e = 8.64, V_max_f = 7.92):
    
    
    #########################################################################################################################
    
    
    """1) Frictional resistance
            - 1st resistance component defined by Holtrop and Mennen (1982)
            - A modification to the original friction line is applied, based on literature of Zeng (2018), to account for shallow water effects """
    C_M = 1.006 - 0.0056 * C_b ** (-3.56)  # Midship section coefficient
    C_wp = (1 + 2 * C_b) / 3  # Waterplane coefficient
    C_p = C_b / C_M  # Prismatic coefficient
    C_BB = 0.2

    delta = C_b * L_wl * B * T  # Water displacement

    lcb = -13.5 + 19.4 * C_p  # longitudinal center of buoyancy
    L_R = L_wl * (1 - C_p + (0.06 * C_p * lcb) / (
                4 * C_p - 1))  # parameter reflecting the length of the run

    A_T = 0.2 * B * T  # transverse area of the transom
    
    A_BT = C_BB * B * T * C_M

    # Total wet area
    S_T = L_wl * (2 * T + B) * np.sqrt(C_M) * (
                0.453 + 0.4425 * C_b - 0.2862 * C_M - 0.003467 * (
                    B / T) + 0.3696 * C_wp) + 2.38 * (A_BT / C_b)

    S_APP = 0.1 * S_T  # Wet area of appendages
    S_B = L_wl * B  # Area of flat bottom

    D_s = 0.7 * T  # Diameter of the screw


    R_e = V_0 * L_wl / nu  # Reynolds number
    D = h - T  # distance from bottom ship to the bottom of the fairway

    # Friction coefficient in deep water
    Cf_0 = 0.075 / ((np.log10(R_e) - 2) ** 2)

    # Friction coefficient proposed, taking into account shallow water effects
    Cf_proposed = (0.08169 / ((np.log10(R_e) - 1.717) ** 2)) * (
                1 + (0.003998 / (np.log10(R_e) - 4.393)) * (D / L_wl) ** (-1.083))

    # 'a' is the coefficient needed to calculate the Katsui friction coefficient
    a = 0.042612 * np.log10(R_e) + 0.56725
    Cf_katsui = 0.0066577 / ((np.log10(R_e) - 4.3762) ** a)

    # The average velocity underneath the ship, taking into account the shallow water effect

    if h / T <= 4:
        V_B = 0.4277 * V_0 * np.exp((h / T) ** (-0.07625))
    else:
        V_B = V_0

    # cf_proposed cannot be applied directly, since a vessel also has non-horizontal wet surfaces that have to be taken
    # into account. Therefore, the following formula for the final friction coefficient 'C_f' is defined:
    C_f = Cf_0 + (Cf_proposed - Cf_katsui) * (S_B / S_T) * (V_B / V_0) ** 2

    # The total frictional resistance R_f [kN]:
    R_f = (C_f * 0.5 * rho * (V_0 ** 2) * S_T) / 1000

    ########################################################################################################################

    """2) Viscous resistance
    - 2nd resistance component defined by Holtrop and Mennen (1982)
    - Form factor (1 + k1) has to be multiplied by the frictional resistance R_f, to account for the effect of viscosity"""

    # c_14 accounts for the specific shape of the afterbody
    c_14 = 1 + 0.0011 * c_stern

    # the form factor (1+k1) describes the viscous resistance
    one_k1 = 0.93 + 0.487118 * c_14 * ((B / L_wl) ** 1.06806) * ((T / L_wl) ** 0.46106) * (
                (L_wl / L_R) ** 0.121563) * (((L_wl ** 3) / delta) ** 0.36486) * (
                              (1 - C_p) ** (-0.60247))

    ########################################################################################################################


    """3) Appendage resistance
    - 3rd resistance component defined by Holtrop and Mennen (1982)
    - Appendages (like a rudder, shafts, skeg) result in additional frictional resistance"""

    # Frictional resistance resulting from wetted area of appendages: R_APP [kN]
    R_APP = (0.5 * rho * (V_0 ** 2) * S_APP * one_k2 * C_f) / 1000

    #################################################################################################################


    """Intermediate calculation: Karpov
    - The Karpov method computes a velocity correction that accounts for limited water depth (corrected velocity V2)
    - V2 has to be implemented in the wave resistance and the residual resistance terms"""

    # The Froude number used in the Karpov method is the depth related froude number F_nh

    # The different alpha** curves are determined with a sixth power polynomial approximation in Excel
    # A distinction is made between different ranges of Froude numbers, because this resulted in a better approximation of the curve
    F_nh = V_0 / np.sqrt(g * h)

    if F_nh <= 0.4:

        if 0 <= h / T < 1.75:
            alpha_xx = (-4 * 10 ** (
                -12)) * F_nh ** 3 - 0.2143 * F_nh ** 2 - 0.0643 * F_nh + 0.9997
        if 1.75 <= h / T < 2.25:
            alpha_xx = -0.8333 * F_nh ** 3 + 0.25 * F_nh ** 2 - 0.0167 * F_nh + 1
        if 2.25 <= h / T < 2.75:
            alpha_xx = -1.25 * F_nh ** 4 + 0.5833 * F_nh ** 3 - 0.0375 * F_nh ** 2 - 0.0108 * F_nh + 1
        if h / T >= 2.75:
            alpha_xx = 1

    if F_nh > 0.4:
        if 0 <= h / T < 1.75:
            alpha_xx = -0.9274 * F_nh ** 6 + 9.5953 * F_nh ** 5 - 37.197 * F_nh ** 4 + 69.666 * F_nh ** 3 - 65.391 * F_nh ** 2 + 28.025 * F_nh - 3.4143
        if 1.75 <= h / T < 2.25:
            alpha_xx = 2.2152 * F_nh ** 6 - 11.852 * F_nh ** 5 + 21.499 * F_nh ** 4 - 12.174 * F_nh ** 3 - 4.7873 * F_nh ** 2 + 5.8662 * F_nh - 0.2652
        if 2.25 <= h / T < 2.75:
            alpha_xx = 1.2205 * F_nh ** 6 - 5.4999 * F_nh ** 5 + 5.7966 * F_nh ** 4 + 6.6491 * F_nh ** 3 - 16.123 * F_nh ** 2 + 9.2016 * F_nh - 0.6342
        if 2.75 <= h / T < 3.25:
            alpha_xx = -0.4085 * F_nh ** 6 + 4.534 * F_nh ** 5 - 18.443 * F_nh ** 4 + 35.744 * F_nh ** 3 - 34.381 * F_nh ** 2 + 15.042 * F_nh - 1.3807
        if 3.25 <= h / T < 3.75:
            alpha_xx = 0.4078 * F_nh ** 6 - 0.919 * F_nh ** 5 - 3.8292 * F_nh ** 4 + 15.738 * F_nh ** 3 - 19.766 * F_nh ** 2 + 9.7466 * F_nh - 0.6409
        if 3.75 <= h / T < 4.5:
            alpha_xx = 0.3067 * F_nh ** 6 - 0.3404 * F_nh ** 5 - 5.0511 * F_nh ** 4 + 16.892 * F_nh ** 3 - 20.265 * F_nh ** 2 + 9.9002 * F_nh - 0.6712
        if 4.5 <= h / T < 5.5:
            alpha_xx = 0.3212 * F_nh ** 6 - 0.3559 * F_nh ** 5 - 5.1056 * F_nh ** 4 + 16.926 * F_nh ** 3 - 20.253 * F_nh ** 2 + 10.013 * F_nh - 0.7196
        if 5.5 <= h / T < 6.5:
            alpha_xx = 0.9252 * F_nh ** 6 - 4.2574 * F_nh ** 5 + 5.0363 * F_nh ** 4 + 3.3282 * F_nh ** 3 - 10.367 * F_nh ** 2 + 6.3993 * F_nh - 0.2074
        if 6.5 <= h / T < 7.5:
            alpha_xx = 0.8442 * F_nh ** 6 - 4.0261 * F_nh ** 5 + 5.313 * F_nh ** 4 + 1.6442 * F_nh ** 3 - 8.1848 * F_nh ** 2 + 5.3209 * F_nh - 0.0267
        if 7.5 <= h / T < 8.5:
            alpha_xx = 0.1211 * F_nh ** 6 + 0.628 * F_nh ** 5 - 6.5106 * F_nh ** 4 + 16.7 * F_nh ** 3 - 18.267 * F_nh ** 2 + 8.7077 * F_nh - 0.4745

        if 8.5 <= h / T < 9.5:
            if F_nh < 0.6:
                alpha_xx = 1
            if F_nh >= 0.6:
                alpha_xx = -6.4069 * F_nh ** 6 + 47.308 * F_nh ** 5 - 141.93 * F_nh ** 4 + 220.23 * F_nh ** 3 - 185.05 * F_nh ** 2 + 79.25 * F_nh - 12.484
        if h / T >= 9.5:
            if F_nh < 0.6:
                alpha_xx = 1
            if F_nh >= 0.6:
                alpha_xx = -6.0727 * F_nh ** 6 + 44.97 * F_nh ** 5 - 135.21 * F_nh ** 4 + 210.13 * F_nh ** 3 - 176.72 * F_nh ** 2 + 75.728 * F_nh - 11.893

    V_2 = V_0 / alpha_xx

    #################################################################################################################


    """4) Wave resistance
    - 4th resistance component defined by Holtrop and Mennen (1982)
    - When the speed or the vessel size increases, the wave making resistance increases
    - In shallow water, the wave resistance shows an asymptotical behaviour by reaching the critical speed"""


    F_n = V_2 / np.sqrt(g * L_wl)  # Froude number

    # parameter c_7 is determined by the B/L ratio
    if B / L_wl < 0.11:
        c_7 = 0.229577 * (B / L_wl) ** 0.33333
    if B / L_wl > 0.25:
        c_7 = 0.5 - 0.0625 * (L_wl / B)
    else:
        c_7 = B / L_wl

    # half angle of entrance in degrees
    i_E = 1 + 89 * np.exp(-((L_wl / B) ** 0.80856) * ((1 - C_wp) ** 0.30484) * (
                (1 - C_p - 0.0225 * lcb) ** 0.6367) * ((L_R / B) ** 0.34574) * (
                                           (100 * delta / (L_wl ** 3)) ** 0.16302))

    c_1 = 2223105 * (c_7 ** 3.78613) * ((T / B) ** 1.07961) * (90 - i_E) ** (-1.37165)
    c_2 = 1  # accounts for the effect of the bulbous bow, which is not present at inland ships
    c_5 = 1 - (0.8 * A_T) / (
                B * T * C_M)  # influence of the transom stern on the wave resistance

    # parameter c_15 depoends on the ratio L^3 / delta
    if (L_wl ** 3) / delta < 512:
        c_15 = -1.69385
    if (L_wl ** 3) / delta > 1727:
        c_15 = 0
    else:
        c_15 = -1.69385 + (L_wl / (delta ** (1 / 3)) - 8) / 2.36

    # parameter c_16 depends on C_p
    if C_p < 0.8:
        c_16 = 8.07981 * C_p - 13.8673 * (C_p ** 2) + 6.984388 * (C_p ** 3)
    else:
        c_16 = 1.73014 - 0.7067

    m_1 = 0.0140407 * (L_wl / T) - 1.75254 * ((delta) ** (1 / 3) / L_wl) - 4.79323 * (
                B / L_wl) - c_16

    m_4 = 0.4 * c_15 * np.exp(-0.034 * (F_n ** (-3.29)))

    if L_wl / B < 12:
        lmbda = 1.446 * C_p - 0.03 * (L_wl / B)
    else:
        lmbda = 1.446 * C_p - 0.036

    # parameters needed for RW_2
    c_17 = 6919.3 * (C_M ** (-1.3346)) * ((delta / (L_wl ** 3)) ** 2.00977) * (
                (L_wl / B - 2) ** 1.40692)
    m_3 = -7.2035 * ((B / L_wl) ** 0.326869) * ((T / B) ** 0.605375)

    ######### When Fn < 0.4
    RW_1 = c_1 * c_2 * c_5 * delta * rho * g * np.exp(
        m_1 * (F_n ** (-0.9)) + m_4 * np.cos(lmbda * (F_n ** (-2))))

    ######## When Fn > 0.5
    RW_2 = c_17 * c_2 * c_5 * delta * rho * g * np.exp(
        m_3 * (F_n ** (-0.9)) + m_4 * np.cos(lmbda * (F_n ** (-2))))

    if F_n < 0.4:
        R_W = RW_1 / 1000  # kN
    if F_n > 0.55:
        R_W = RW_2 / 1000  # kN
    else:
        R_W = (RW_1 + ((10 * F_n - 4) * (RW_2 - RW_1)) / 1.5) / 1000  # kN

    #################################################################################################################

    """5) Bulbous Bow Resistance terms
    - Extra resistance due to bulbousbow. Not for IWT, Yes for Dredging Vessels"""
    
    # R_B
    
    
    # parameters
    T_F = T    #Forward draft of the vessel [m]
    h_B = 0.2 * T    #Position of the centre of the transverse area [m]
    
    
    # Froude number based on immersoin of bulbous bow [-]
    F_ni = (V_2/np.sqrt(g*(T_F - h_B - 0.25*np.sqrt(A_BT)+0.15*V_2**2)))
    
    #P_B is coefficient for the emergence of bulbous bow
    P_B = (0.56 * np.sqrt(A_BT))/(T_F - 1.5*h_B)
    
    R_B = ((0.11 * np.exp(-3* P_B**2) * F_ni**3 * A_BT**1.5 * rho * g)/(1+ F_ni**2)) /1000
    #################################################################################################################
    """6) Residual resistance terms
    - Holtrop and Mennen (1982) defined three residual resistance terms:
    - 1) Resistance due to the bulbous bow (not incorporated since inland ships in general don't have a bulb)
    - 2) Resistance due to immersed transom
    - 3) Resistance due to model-ship correlation """

    # Resistance due to immersed transom: R_TR [kN]
    F_nt = V_2 / np.sqrt(
        2 * g * A_T / (B + B * C_wp))  # Froude number based on transom immersion
    
    if F_nt < 5:
        c_6 = 0.2 * (1- 0.2 * F_nt)
    else:
        c_6 = 0
        
    
    R_TR = (0.5 * rho * (V_2 ** 2) * A_T * c_6) / 1000

    # Model-ship correlation resistance: R_A [kN]

    if T / L_wl < 0.04:
        c_4 = T / L_wl
    else:
        c_4 = 0.04
    c_2 = 1

    C_A = 0.006 * (L_wl + 100) ** (-0.16) - 0.00205 + 0.003 * np.sqrt(L_wl / 7.5) * (
                C_b ** 4) * c_2 * (0.04 - c_4)

    ####### Holtrop and Mennen in the document of Sarris, 2003 #######
    R_A = (0.5 * rho * (V_2 ** 2) * S_T * C_A) / 1000  # kW

    #################################################################################################################

    R_tot = R_f*one_k1 + R_APP + R_W + R_B + R_TR + R_A
    
    print('my total resistance = {:.2f} kN'.format(R_tot))
    print('my f resistance = {:.2f} kN'.format(R_f))
    print('my app resistance = {:.2f} kN'.format(R_APP))
    print('my w resistance = {:.2f} kN'.format(R_W))
    print('my b resistance = {:.2f} kN'.format(R_B))
    print('my tr resistance = {:.2f} kN'.format(R_TR))
    print('my a resistance = {:.2f} kN'.format(R_A))
    
    if V_0 > V_max_e:
        P_empty = (R_tot * V_max_e)
    else:
        P_empty = (R_tot * V_0)
    
    if V_0 > V_max_f:
        P_full = R_tot * V_max_f * 1.2
    else:
        P_full = R_tot * V_0 * 1.2
    
    
    P_board = 0.05 * P_inst
    
    P_tot_e = P_empty + P_board
    P_tot_f = P_full + P_board
    print('my P_tot_empty = {:.2f} kW'.format(P_tot_e))
    print('my T = {:.2f} m'.format(T))
   
    
    return P_tot_e

In [4]:
class HasEnergyPlugin:
    """Mixin for Activity to initialize WeatherPluginActivity."""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        energy_plugin = EnergyPlugin()
        self.register_plugin(plugin=energy_plugin, priority=3)


class EnergyPlugin(model.AbstractPluginClass):
    """Mixin for all activities to add delay and downtime."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        
    def post_process(
        self, env, activity_log, activity, start_activity, *args, **kwargs
    ):
        print('')
        print('post_process: {}, {}'.format(activity.mover.name,activity.name))
        if activity.name == 'sailing empty':
            V_0 = activity.mover.compute_ve(None)
        else:
            V_0 = activity.mover.compute_vf(None)

        print('my velocity is {} m/s'.format(V_0))
        
        if activity.name == "sailing empty":
            T = activity.mover.T_e
        else:
            T = activity.mover.T_f
        self.power_empty = calculate_power(V_0=V_0,T=activity.mover.T_e)
        self.power_full = calculate_power(V_0=V_0, T=activity.mover.T_f)
        if activity.name == 'sailing empty':
            power = self.power_empty
        else:
            power = self.power_full
        activity_duration = env.now - start_activity
        activity.log_entry(
            t=env.now,
            activity_id=activity.id,
            activity_state=core.LogState.UNKNOWN,
            additional_state={
                "power":power,
                "activity_duration":activity_duration,
                "energy_used":power*activity_duration,
            }
        )
        
        return {}

#### 1. Initialise simpy environment

In [5]:
# setup environment (simulation time needs to match the available weather data)
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes

In [6]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
        opentnsim.core.VesselProperties
    ),
    {},
)

#### 3. Create objects
##### 3.1. Create site object(s)

In [7]:
# prepare input data for from_site
# 0.8983 = 100km
location_from_site = shapely.geometry.Point(0, 0)
data_from_site = {"env": my_env,
                  "name": "from_site",
                  "geometry": location_from_site,
                  "capacity": 12,
                  "level": 12
                 }
# instantiate from_site 
from_site = Site(**data_from_site)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(0.16636516, 0)
data_to_site = {"env": my_env,
                "name": "to_site",
                "geometry": location_to_site,
                "capacity": 12,
                "level": 0
               }
# instantiate to_site 
to_site = Site(**data_to_site)

##### 3.2. Create vessel object(s)

In [8]:
# prepare input data for vessel_01
data_vessel01 = {"env": my_env,
                 "name": "vessel01",
                 "geometry": location_from_site, 
                 "loading_rate": 1,
                 "unloading_rate": 1,
                 "capacity": 4,
                 "compute_v": lambda x: 5,
                  "type":None,
                 "B":32,
                 "L":159,
                 "h_min":20,
                 "T":13.55,
#                  "T_e":1
                  
               }
# instantiate vessel_01 
vessel01 = TransportProcessingResource(**data_vessel01)

# Monkeypatch with L: Data from EIS HAM318
# Man Jiang case: 6 m waterdepth, M8 vessel at 4.0 m/s
vessel01.h = 70       # m waterdepth 
vessel01.L_wl = 159     # m length waterline 
vessel01.L_oa = 225       #m Lentgh overall
vessel01.B = 32    #m beam 
vessel01.T_e = 5.9 #m draft empty
vessel01.T_f = 13.55   #m draft full
vessel01.compute_ve = lambda x: 8.64 # m/s
vessel01.compute_vf = lambda x: 7.92 # m/s
vessel01.ukc=4

# # Ham318 case
# vessel01.h = 10      # m waterdepth
# vessel01.L = 225.71  # m length (should be L_wl ... now typically L_oa)
# vessel01.B = 32.046  # m beam
# vessel01.T = 13.55   # m draught

##### 3.3 Create activity/activities

In [9]:
# initialise registry
registry = {}
keep_resources = {}

In [10]:
# create delay sequence activities
DelaySequenceActivity =  type(
    "TestShiftActivity",
    (
        model.SequentialActivity,  # the order is critical!
    ),
    {},
)

DelayWhileActivity =  type(
    "TestShiftActivity",
    (
        model.WhileActivity,  # the order is critical!
    ),
    {},
)

DelayMoveActivity =  type(
    "TestMoveActivity",
    (
        HasEnergyPlugin,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)

DelayShiftActivity =  type(
    "TestShiftActivity",
    (
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

EnergyBasicActivity =  type(
    "TestShiftActivity",
    (
        model.BasicActivity,  # the order is critical!
    ),
    {},
)

In [11]:
# create a list of the sub processes
sub_processes = [
    DelayMoveActivity(
        env=my_env,
        name="sailing empty",
        registry=registry,
        mover=vessel01,
        destination=from_site,
    ),
    DelayShiftActivity(
        env=my_env,
        name="load cargo",
        registry=registry,
        processor=vessel01,
        origin=from_site,
        destination=vessel01,
        amount=4,
        duration=3600,
    ),
    DelayMoveActivity(
        env=my_env,
        name="sailing full",
        registry=registry,
        mover=vessel01,
        destination=to_site,
    ),
    DelayShiftActivity(
        env=my_env,
        name="unload cargo",
        registry=registry,
        processor=vessel01,
        origin=vessel01,
        destination=to_site,
        amount=4,
        duration=3600,
    ),
    EnergyBasicActivity(
        env=my_env,
        name="Basic activity",
        registry=registry,
        duration=0,
        additional_logs=[vessel01],
    ),
]

# create a 'sequential activity' that is made up of the 'sub_processes'
sequential_activity = DelaySequenceActivity(
    env=my_env,
    name="Single run process",
    registry=registry,
    sub_processes=sub_processes,
)

# create a while activity that executes the 'sequential activity' while a stop condition is not met 
while_activity = DelayWhileActivity(
    env=my_env,
    name="While activity",
    registry=registry,
    sub_processes=[sequential_activity],
    condition_event=[{"type": "container", "concept": to_site, "state": "full"}],
)

In [12]:
vessel01.__dict__

{'type': None,
 'B': 32,
 'L': 159,
 '_T': 13.55,
 '_h_min': 20,
 'H_e': None,
 'H_f': None,
 'T_e': 5.9,
 'T_f': 13.55,
 'unloading_rate': 1,
 'unload_manoeuvring': 0,
 'loading_rate': 1,
 'load_manoeuvring': 0,
 'geometry': <shapely.geometry.point.Point at 0x1efff05ca00>,
 'wgs84': Geod(ellps='WGS84'),
 'env': <simpy.core.Environment at 0x1efff05dcc0>,
 'resource': <simpy.resources.resource.Resource at 0x1efff05d090>,
 'container': <openclsim.core.events_container.EventsContainer at 0x1efff05c280>,
 'v': 1,
 'compute_v': <function __main__.<lambda>(x)>,
 'log': {'Timestamp': [],
  'ActivityID': [],
  'ActivityState': [],
  'ObjectState': [],
  'ActivityLabel': []},
 'name': 'vessel01',
 'id': 'fa1f1d31-cc17-4742-9954-2a8851d82a8d',
 'h': 70,
 'L_wl': 159,
 'L_oa': 225,
 'compute_ve': <function __main__.<lambda>(x)>,
 'compute_vf': <function __main__.<lambda>(x)>,
 'ukc': 4}

#### 4. Register processes and run simpy

In [13]:
model.register_processes([while_activity])
my_env.run()


post_process: vessel01, sailing empty
my velocity is 8.64 m/s
my total resistance = 2381.19 kN
my f resistance = 339.91 kN
my app resistance = 84.98 kN
my w resistance = 1565.29 kN
my b resistance = 8.07 kN
my tr resistance = 145.90 kN
my a resistance = 101.46 kN
my P_tot_empty = 22005.27 kW
my T = 5.90 m
my total resistance = 4519.66 kN
my f resistance = 470.61 kN
my app resistance = 117.65 kN
my w resistance = 3082.72 kN
my b resistance = 62.26 kN
my tr resistance = 446.80 kN
my a resistance = 133.54 kN
my P_tot_empty = 40481.67 kW
my T = 13.55 m

post_process: vessel01, sailing full
my velocity is 7.92 m/s
my total resistance = 1503.82 kN
my f resistance = 288.70 kN
my app resistance = 72.17 kN
my w resistance = 802.25 kN
my b resistance = 7.69 kN
my tr resistance = 132.61 kN
my a resistance = 85.25 kN
my P_tot_empty = 13342.09 kW
my T = 5.90 m
my total resistance = 2811.47 kN
my f resistance = 399.69 kN
my app resistance = 99.92 kN
my w resistance = 1577.51 kN
my b resistance = 56

#### 5. Inspect results
##### 5.1 Inspect logs

In [14]:
df = pd.concat(
    [
        plot.get_log_dataframe(act, [while_activity, sequential_activity, *sub_processes])
        for act in sub_processes
    ]
).sort_values(by=['Timestamp'])
df

,Activity,Timestamp,ActivityState,power,activity_duration,energy_used
0,sailing empty,1970-01-01 00:00:00.000000,START,NaN,NaN,NaN
1,sailing empty,1970-01-01 00:00:00.000000,STOP,NaN,NaN,NaN
2,sailing empty,1970-01-01 00:00:00.000000,UNKNOWN,22005.274947,0.000000,0.000000e+00
0,load cargo,1970-01-01 00:00:00.000000,START,NaN,NaN,NaN
0,sailing full,1970-01-01 01:00:00.000000,START,NaN,NaN,NaN
1,load cargo,1970-01-01 01:00:00.000000,STOP,NaN,NaN,NaN
0,unload cargo,1970-01-01 02:01:43.936979,START,NaN,NaN,NaN
1,sailing full,1970-01-01 02:01:43.936979,STOP,NaN,NaN,NaN
2,sailing full,1970-01-01 02:01:43.936979,UNKNOWN,23698.669779,3703.936979,8.777838e+07
0,Basic activity,1970-01-01 03:01:43.936979,START,NaN,NaN,NaN


In [15]:
df['energy_used'].sum()

426347441.3072189

In [16]:
def calculate_actual_T_and_payload(self, h_min, ukc=4.0,vesl_type="Dry_DH"):
        """ Calculate actual draft based on Van Dorsser et al
        https://www.researchgate.net/publication/344340126_The_effect_of_low_water_on_loading_capacity_of_inland_ships
        """
        #Design draft T_design, refer to Table 5
#         ukc = 4 m for open sea (EIS)

        Tdesign_coefs = dict({"intercept":0,
                         "c1": 1.7244153371,
                         "c2": 2.2767179246,
                         "c3": 1.3365379898,
                         "c4": -5.9459308905,
                         "c5": 6.2902305560*10**-2,
                         "c6": 7.7398861528*10**-5,
                         "c7": 9.0052384439*10**-3,
                         "c8": 2.8438560877
                         })

        assert vesl_type in ["Container","Dry_SH","Dry_DH","Barge","Tanker"],'Invalid value vesl_type, should be "Container","Dry_SH","Dry_DH","Barge" or "Tanker"'
        if vesl_type == "Container":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [1,0,0,0]
        elif vesl_type == "Dry_SH":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,1,0,0]
        elif vesl_type == "Dry_DH":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,1,0,0]
        elif vesl_type == "Barge":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,1,0]

        elif vesl_type == "Tanker":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,0,1]

        T_design = Tdesign_coefs['intercept'] + (dum_container * Tdesign_coefs['c1']) + \
                                                (dum_dry * Tdesign_coefs['c2']) + \
                                                (dum_barge * Tdesign_coefs['c3']) +\
                                                (dum_tanker * Tdesign_coefs['c4']) +\
                                                (Tdesign_coefs['c5'] * dum_container * self.L_oa**0.4 * self.B**0.6) +\
                                                (Tdesign_coefs['c6'] * dum_dry * self.L_oa**0.7 * self.B**2.6)+\
                                                (Tdesign_coefs['c7'] * dum_barge * self.L_oa**0.3 * self.B**1.8) +\
                                                (Tdesign_coefs['c8'] * dum_tanker * self.L_oa**0.1 * self.B**0.3)

        #Empty draft T_empty, refer to Table 4
        Tempty_coefs = dict({"intercept": 7.5740820927*10**-2,
                    "c1": 1.1615080992*10**-1,
                    "c2": 1.6865973494*10**-2,
                    "c3": -2.7490565381*10**-2,
                    "c4": -5.1501240744*10**-5,
                    "c5": 1.0257551153*10**-1,
                    "c6": 2.4299435211*10**-1,
                    "c7": -2.1354295627*10**-1,
                    })


        if vesl_type == "Container":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [1,0,0,0]
        elif vesl_type == "Dry_SH":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,0,0]
        elif vesl_type == "Dry_DH":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,1,0,0]
        elif vesl_type == "Barge":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,1,0]

        elif vesl_type == "Tanker":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,0,1]

        # dum_container and dum_dry use the same "c5"
        T_empty = Tempty_coefs['intercept']  + (Tempty_coefs['c1'] * self.B) + \
                                               (Tempty_coefs['c2'] * ((self.L_oa * T_design) / self.B)) + \
                                               (Tempty_coefs['c3'] * (np.sqrt(self.L_oa * self.B)))  + \
                                               (Tempty_coefs['c4'] * (self.L_oa * self.B * T_design)) +  \
                                               (Tempty_coefs['c5'] * dum_container) + \
                                               (Tempty_coefs['c5'] * dum_dry)   + \
                                               (Tempty_coefs['c6'] * dum_tanker) + \
                                               (Tempty_coefs['c7'] * dum_barge)

        #Actual draft T_actual
        if (T_design <= (h_min - ukc)):
            T_actual = T_design

        elif T_empty > (h_min - ukc):
            T_actual =  (f"No trip possible. Available depth smaller than empty draft: {h_min - T_empty} m")

        elif (T_design > (h_min - ukc)):
            T_actual = h_min -  ukc

        
        #Capacity indexes, refer to Table 3 and eq 2
        CI_coefs = dict({"intercept": 2.0323139721 * 10**1,

                "c1": -7.8577991460 * 10**1,
                "c2": -7.0671612519 * 10**0,
                "c3": 2.7744056480 * 10**1,
                "c4": 7.5588609922 * 10**-1,
                "c5": 3.6591813315 * 10**1
                })
        # Capindex_1 related to actual draft (especially used for shallow water)
        Capindex_1 = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2)  +  (
        CI_coefs["c3"] * T_actual) + (CI_coefs["c4"] * T_actual**2)   + ( CI_coefs["c5"] * (T_empty * T_actual))
        # Capindex_2 related to design draft
        Capindex_2 = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2)   + (
        CI_coefs["c3"] * T_design) + (CI_coefs["c4"] * T_design**2)  + (CI_coefs["c5"] * (T_empty * T_design))

        #DWT design capacity, refer to Table 6 and eq 3
        capacity_coefs = dict({"intercept": -1.6687441313*10**1,
             "c1": 9.7404521380*10**-1,
             "c2": -1.1068568208,
             })

        DWT_design = capacity_coefs['intercept'] + (capacity_coefs['c1'] * self.L_oa * self.B * T_design) + (
         capacity_coefs['c2'] * self.L_oa * self.B * T_empty) # designed DWT
        DWT_actual = (Capindex_1/Capindex_2)*DWT_design # actual DWT of shallow water


        if T_actual < T_design:
            consumables=0.04 #consumables represents the persentage of fuel weight,which is 4-6% of designed DWT
                              # 4% for shallow water (Van Dosser  et al. Chapter 8,pp.68).
        else:
            consumables=0.06 #consumables represents the persentage of fuel weight,which is 4-6% of designed DWT
                              # 6% for deep water (Van Dosser et al. Chapter 8, pp.68).

        fuel_weight=DWT_design*consumables #(Van Dosser et al. Chapter 8, pp.68).
        actual_max_payload = DWT_actual-fuel_weight # payload=DWT-fuel_weight
        
        print('T_actual = {:.2f} m'.format(T_actual))
        print('T_empty = {:.2f} m'.format(T_empty))
        print('T_design = {:.2f} m'.format(T_design))
        print('actual max payload = {:.2f} kg'.format(actual_max_payload))
        
        return T_actual, actual_max_payload, T_empty, T_design



In [17]:
x = calculate_actual_T_and_payload(vessel01, 20, ukc=.3,vesl_type="Dry_DH")


T_actual = 19.70 m
T_empty = -6.10 m
T_design = 30.37 m
actual max payload = 163168.21 kg


##### 5.2 Visualise gantt charts

In [18]:
objects = [while_activity, sequential_activity, vessel01, from_site, to_site]
objects.extend(sub_processes)
plot.get_gantt_chart(objects)

In [19]:
# plotting just the vessel activities ()
objects = [vessel01]
objects.extend(sub_processes)
plot.get_gantt_chart(objects)

In [20]:
print(sub_processes)

[<__main__.TestMoveActivity object at 0x000001EFFF05F3A0>, <__main__.TestShiftActivity object at 0x000001EFFF05D0C0>, <__main__.TestMoveActivity object at 0x000001EFFF05EDA0>, <__main__.TestShiftActivity object at 0x000001EFFF05E350>, <__main__.TestShiftActivity object at 0x000001EFFF05E4D0>]
